In [ ]:
#| default_exp display

In [ ]:
#| export
from __future__ import annotations


# Display
> Rich display utilities and dynamic CSS for Jupyter notebooks

This module enhances Jupyter's display capabilities with:

- Rich rendering — Leverage `rich` library for beautiful object display
- Interactive JSON — Collapsible JSON viewer with configurable depth
- Dynamic CSS — Generate and update CSS stylesheets programmatically
- Global styles — Manage notebook-wide CSS that updates reactively

## Key Components

- `pretty_repr` — Render objects using `rich` formatting (HTML + text)
- `rich_display` — Display objects with rich formatting and updateable handles
- `RenderJSON` — Interactive collapsible JSON viewer
- `cssmap` — Convert Python dicts to CSS strings (including nested at-rules)
- `GlobalCSS` — Manage global CSS with reactive updates

## When to Use

- Pretty printing — Display complex data structures with syntax highlighting
- Data exploration — Inspect nested JSON/dicts with collapsible views
- Notebook styling — Generate CSS from Python for consistent theming
- Dynamic UIs — Update styles reactively based on state changes

<!-- # Prologue -->

In [ ]:
#| export
import json
import uuid
from typing import Mapping
from typing import overload
from typing import Sequence
from typing import TypeAlias

import fastcore.all as FC
import rich
from IPython.display import display
from IPython.display import DisplayHandle
from IPython.display import HTML


In [ ]:
#| hide
from fastcore.test import *
from pote.common import setup_console
from pote.test import *
from rich.markdown import Markdown


In [ ]:
#| hide
console, cprint = setup_console(140)

# Rich display

Utilities for rendering objects with `rich` formatting in Jupyter. These functions generate both HTML (for rich notebooks) and plain text (for terminal/markdown) representations.

**Key features**:
- Syntax highlighting for data structures
- Configurable output formats (HTML, text, or both)
- Updateable display handles for reactive UIs

In [ ]:
from rich.json import JSON
from rich.jupyter import display as rich_display

In [ ]:
def display_json(json):
    json_renderable = JSON.from_data(json)
    a = list(console.render(json_renderable))
    rich_display(a, console._render_buffer(a))

In [ ]:
display_json({'a': 1, 'b': 2})

{
  "a": 1,
  "b": 2
}

In [ ]:
#| export

@overload
def pretty_repr(*o, html:bool=True, text:bool=False, **kwargs) -> str: ...
@overload
def pretty_repr(*o, html:bool=False, text:bool=True, **kwargs) -> str: ...
def pretty_repr(*o, html:bool=True, text:bool=True, **kwargs) -> dict[str, str]|str:
    from rich.pretty import Pretty
    d = Pretty(*o, **kwargs)._repr_mimebundle_(
        include=((),('text/plain',))[text] + ((),('text/html',))[html], 
        exclude=((),('text/plain',))[not text] + ((),('text/html',))[not html]
        )
    return d if len(d) > 1 else tuple(d.values())[0]

In [ ]:
display(HTML(pretty_repr({'a': 1, 'b': [1,2,3]}, text=False)))
print(pretty_repr({'a': 1, 'b': [1,2,3]}, html=False))
cprint({'a': 1, 'b': [1,2,3]})

{'a': 1, 'b': [1, 2, 3]}



{'a': 1, 'b': [1, 2, 3]}

In [ ]:
#| export

def rich_display(*o, dhdl: DisplayHandle|None=None):
    if not o: return
    vv:tuple[str, ...] = tuple(FC.flatten([_.items() for _ in map(pretty_repr, o)]))  # type: ignore
    dd = {'text/plain':'\n'.join(vv[1::4]), 'text/html':'\n'.join(vv[3::4])}
    if dhdl: dhdl.update(dd, raw=True)
    else: display(dd, raw=True)

In [ ]:
rich_display({'a': 1, 'b': 2}, [3, 4, 5])

{'a': 1, 'b': 2}

[3, 4, 5]

In [ ]:
dhdl = display(display_id=True)
rich_display({'a': 1, 'b': 2}, [3, 4, 5], dhdl=dhdl)


{'a': 1, 'b': 2}

[3, 4, 5]

# Collapsible JSON

Interactive JSON viewer using [renderjson](https://github.com/caldwell/renderjson). Displays JSON with collapsible nodes, perfect for exploring deeply nested data.

**Features**:
- Click to expand/collapse nodes
- Configurable initial expansion level
- Scrollable container with max height
- Automatic CDN loading of renderjson library

**Use cases**: API responses, config files, deeply nested data structures

In [ ]:
#| export

class RenderJSON(object):
    def __init__(self, json_data, max_height=200, init_level=0):
        if isinstance(json_data, Sequence):
            s = json.dumps(list(json_data))
        elif isinstance(json_data, Mapping):
            s = json.dumps(dict(json_data))
        elif hasattr(json_data, 'to_dict'):
            s = json.dumps(json_data.to_dict())
        elif hasattr(json_data, 'to_json'):
            s = json_data.to_json()
        else:
            s = json_data
        self.json_str = s
        self.uuid = str(uuid.uuid4())
        self.max_height = max_height
        self.init_level = init_level

    def display(self):
        html_content = f"""
<div id="wrapper-{self.uuid}" style="width: 100%; max-height: {self.max_height}px; overflow-y: auto;">
    <div id="{self.uuid}" style="width: 100%;"></div>
    <script>
        function renderMyJson() {{
            renderjson.set_show_to_level({self.init_level});
            document.getElementById('{self.uuid}').appendChild(renderjson({self.json_str}));
        }};
        function loadRenderjson() {{
            if (window.renderjson) return Promise.resolve();
            return new Promise((resolve, reject) => {{
                const script = document.createElement('script');
                script.src = 'https://cdn.jsdelivr.net/npm/renderjson@latest/renderjson.js';
                script.onload = resolve;
                script.onerror = reject;
                document.head.appendChild(script);
            }});
        }};
        loadRenderjson().then(renderMyJson).catch(err => console.error('Failed to load renderjson:', err));
    </script>
</div>
"""
        display(HTML(html_content), metadata={'bridge': {'skip':True}})

    def _ipython_display_(self):
        self.display()

In [ ]:
json_data = {
    "name": "Petronila",
    "age": 28,
    "interests": ["reading", "cycling", "technology"],
    "education": {
        "bachelor": "Computer Science",
        "master": "Data Science",
        "phd": "Not enrolled"
    }
}

RenderJSON(json_data, init_level=1).display()

# CSS

Utilities for generating and managing CSS in notebooks. Write styles in Python dicts instead of CSS strings for better maintainability and dynamic generation.

## cssmap

Convert Python dictionaries to CSS strings. Supports:
- Nested selectors
- At-rules (`@media`, `@scope`, `@keyframes`, etc.)
- Proper indentation and formatting

**Why use this?**
- Type-safe CSS generation
- Easier to compose and reuse styles
- Programmatic style generation (loops, conditionals)

In [ ]:
#| export

selector: TypeAlias = str
at_rule: TypeAlias = str
ruleset: TypeAlias = dict[str, str]

@overload
def cssmap(stylesheet: dict[selector, ruleset], lvl: int = 0) -> str: ...

@overload
def cssmap(stylesheet: dict[at_rule, dict[selector, ruleset]], lvl: int = 0) -> str: ...

def cssmap(stylesheet, lvl: int = 0) -> str:
    def format_ruleset(rset: ruleset) -> str: 
        indent = '  ' * (lvl + 1)
        return ';\n'.join(f'{indent}{k}: {v}' for k, v in rset.items())
    
    def format_block(sel: str, content: str) -> str: 
        indent = '  ' * lvl
        return f'{indent}{sel} {{\n{content}\n{indent}}}'
    
    css_blocks = [
        f'{selector} {{\n{cssmap(rules, lvl+1)}\n{" "*lvl}}}' 
        if selector.startswith('@') else 
        format_block(selector, format_ruleset(rules))
        for selector, rules in stylesheet.items()
    ]
    
    return '\n\n'.join(css_blocks)

In [ ]:
expected = '''@scope (div.tableFixHead) {
  :scope {
    overflow-y: auto;
    max-width: max-content
  }

  thead th {
    position: sticky;
    top: 0px;
    background-color: gray
  }
}

body {
  font-family: Arial, sans-serif;
  margin: 0;
  padding: 20px
}'''

In [ ]:

style = {
    '@scope (div.tableFixHead)': {
        ':scope': {
            'overflow-y': 'auto',
            'max-width': 'max-content',
        },
        'thead th': {
            'position': 'sticky',
            'top': '0px',
            'background-color': 'gray',
        }
    },
    'body': {
        'font-family': 'Arial, sans-serif',
        'margin': '0',
        'padding': '20px'
    }
}


css = cssmap(style)
test_eq(css, expected)
display(Markdown(f"```css\n{css}\n```"))

                                                                                                                   
 @scope (div.tableFixHead) {                                                                                       
   :scope {                                                                                                        
     overflow-y: auto;                                                                                             
     max-width: max-content                                                                                        
   }                                                                                                               
                                                                                                                   
   thead th {                                                                                                      
     position: sticky;                                                                                             
     top: 0px;                                                                                                     
     background-color: gray                                                                                        
   }                                                                                                               
 }                                                                                                                 
                                                                                                                   
 body {                                                                                                            
   font-family: Arial, sans-serif;                                                                                 
   margin: 0;                                                                                                      
   padding: 20px                                                                                                   
 }                                                                                                                 
                                                                                                                   


## GlobalCSS

Manage notebook-wide CSS with reactive updates. Create a single `GlobalCSS` instance, display it once, then add/update styles dynamically throughout your notebook.

**Pattern**:
```python
# Cell 1: Initialize and display
GCSS = GlobalCSS()
display(GCSS)

# Cell 2: Add styles dynamically
GCSS.add('my-component', '.my-component { color: blue; }')

# Cell 3: Update existing styles
GCSS.update('my-component', '.my-component { color: red; }')
```

**Benefits**:
- Single source of truth for notebook styles
- No duplicate `<style>` tags
- Reactive updates without re-executing cells
- Named styles for easy management

In [ ]:
#| export

class GlobalCSS:
    _style_tmpl = '<style id="{name}">{css}</style>'
    def _html_widget(self, name:str, css:str): return self._style_tmpl.format(name=name, css=css)
    def has_style(self, name:str): return name in self._name2n
    @property
    def css(self): return '\n'.join(self._css)

    def add(self, name:str, css, update: bool=False):
        if self.has_style(name):
            if not update: return
        else:
            n = len(self._css)
            self._name2n[name] = n
            self._css.append('')
        self.update(name, css)

    def update(self, name:str, css):
        css = css if isinstance(css, str) else FC.valmap(css)
        if css and (n := self._name2n.get(name)) is not None:
            self._css[n] = self._html_widget(name=name, css=css)
            if self._dh:
                self._dh.update(HTML(self.css))
    
    _dh: DisplayHandle
    def display(self): 
        if self._dh is None: self._dh = display(HTML(self.css), display_id=True)  # type: ignore
        else: print('GlobalCSSs should be displayed only once, skipping.')
    def _ipython_display_(self): self.display()

    def __init__(self, css:dict[str, str] | None = None):
        self._ui_done = False
        self._dh = None  # type: ignore
        css = css or {}
        self._name2n = {name: n for n,name in enumerate(css.keys())}
        self._css = [self._html_widget(name, css) for name,css in css.items()]

In [ ]:
GCSS = GlobalCSS()
display(GCSS)

In [ ]:
%%HTML
<div class="__test__gcss__">Hello</div>

In [ ]:
GCSS.add('test', '.__test__gcss__ { color: red; }')

In [ ]:
GCSS.update('test', '.__test__gcss__ { color: green; }')

In [ ]:
test_stdout(lambda: display(GCSS), 'GlobalCSSs should be displayed only once, skipping.')

----
<!-- # Colophon -->

In [ ]:
#|hide
#|eval: false

import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean

In [ ]:
#|hide
#|eval: false

if FC.IN_NOTEBOOK:
    nb_path = '17_display.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)